<h1> image to hex

In [1]:
import numpy as np
from PIL import Image

# 1. 이미지 읽기 및 RGB 변환
image = Image.open("dog.jpg").convert("RGB")
image_data = np.array(image)  # RGB 픽셀 데이터 (H, W, 3)

# 2. RGB 값을 0~1 사이로 정규화
image_data = image_data / 255.0  # 0~1 범위로 변환

# 3. 고정 소수점 변환 (sign(1), int(7), real(8))
# 16비트 고정 소수점 형식으로 변환
def float_to_fixed(value, int_bits, frac_bits):
    scale = 2 ** frac_bits
    fixed_value = np.round(value * scale).astype(np.int16)  # 16비트 정수
    return np.clip(fixed_value, -(2**(int_bits+frac_bits-1)), 2**(int_bits+frac_bits-1)-1)

# 각 채널(R, G, B)을 고정 소수점으로 변환
fixed_image_data = np.zeros_like(image_data, dtype=np.int16)
for channel in range(3):  # R, G, B 각각 처리
    fixed_image_data[..., channel] = float_to_fixed(image_data[..., channel], 7, 8)

# 4. 결과 저장 (옵션)
np.save("dog_fixed16.npy", fixed_image_data)  # 저장 (사용자가 불러올 수 있도록)


<h1> restore image

In [2]:
import numpy as np
from PIL import Image

# 1. 저장된 고정 소수점 데이터 불러오기
fixed_image_data = np.load("dog_fixed16.npy")  # (H, W, 3)

# 2. 고정 소수점 데이터를 부동 소수점으로 복원
def fixed_to_float(fixed_value, frac_bits):
    scale = 2 ** frac_bits
    return fixed_value / scale

# R, G, B 채널 복원
frac_bits = 8
restored_image_data = np.zeros_like(fixed_image_data, dtype=np.float32)
for channel in range(3):
    restored_image_data[..., channel] = fixed_to_float(fixed_image_data[..., channel], frac_bits)

# 3. 복원된 데이터를 [0, 255] 범위로 변환
restored_image_data = np.clip(restored_image_data * 255, 0, 255).astype(np.uint8)

# 4. 이미지를 저장 및 표시
restored_image = Image.fromarray(restored_image_data, mode="RGB")
restored_image.save("restored_dog.jpg")
restored_image.show()  # 화면에 표시


<h1> open numpy file

In [5]:
import numpy as np
from PIL import Image

# 1. numpy 파일 로드
fixed_image_data = np.load("dog_fixed16.npy")  # (H, W, 3)

# 2. 원본 이미지 데이터 로드
original_image = np.array(Image.open("dog.jpg").convert("RGB"))  # (H, W, 3)

# 3. 16비트 고정 소수점 값을 hex로 변환
def fixed_to_hex(value):
    """16비트 정수 값을 hex로 변환"""
    return f"0x{value & 0xFFFF:04X}"  # 16비트로 변환 후 4자리 hex 표현

# 4. 텍스트 파일로 저장
h, w, c = fixed_image_data.shape
with open("dog_fixed16_with_hex_and_original.txt", "w") as f:
    f.write("Hex(Fixed_Point R, G, B)    # Original(R, G, B)\n")  # 헤더 작성
    
    for i in range(h):
        for j in range(w):
            # 고정 소수점 값 (16비트)
            fixed_r_hex = fixed_to_hex(fixed_image_data[i, j, 0])
            fixed_g_hex = fixed_to_hex(fixed_image_data[i, j, 1])
            fixed_b_hex = fixed_to_hex(fixed_image_data[i, j, 2])
            
            # 원본 RGB 값
            orig_r, orig_g, orig_b = original_image[i, j]
            
            # 라인 작성
            f.write(f"({fixed_r_hex}, {fixed_g_hex}, {fixed_b_hex})    # ({orig_r}, {orig_g}, {orig_b})\n")

print("고정 소수점 값(16비트 hex)과 원래 RGB 값이 'dog_fixed16_with_hex_and_original.txt'로 저장되었습니다.")


고정 소수점 값(16비트 hex)과 원래 RGB 값이 'dog_fixed16_with_hex_and_original.txt'로 저장되었습니다.
